In [1]:
import pandas as pd
import os

In [2]:
list_csv = list(map(lambda txtMap: txtMap.replace(".csv", ""), filter(lambda txt: txt.endswith(".csv"), os.listdir("data-brazil"))))
list_csv

['appearances',
 'game_events',
 'game_lineups',
 'players',
 'player_valuations',
 'transfers']

In [3]:
df_appearances = pd.read_csv("data-brazil/appearances.csv")
df_game_events = pd.read_csv("data-brazil/game_events.csv")
df_game_lineups = pd.read_csv("data-brazil/game_lineups.csv")
df_players = pd.read_csv("data-brazil/players.csv")
df_player_valuations = pd.read_csv("data-brazil/player_valuations.csv")
df_transfers = pd.read_csv("data-brazil/transfers.csv")
##
df_games = pd.read_csv("data-new/games.csv")
df_competitions = pd.read_csv("data-new/competitions.csv")
df_clubs = pd.read_csv("data-new/clubs.csv")
df_clubs_games = pd.read_csv("data-new/club_games.csv")
df_games = df_games[['game_id', 'competition_id', 'season', 'round', 'date', 'home_club_id','away_club_id', 'home_club_goals', 'away_club_goals','home_club_manager_name', 'away_club_manager_name', 'referee', 'home_club_formation', 'away_club_formation', 'aggregate','competition_type']]

C:\Users\Admin\AppData\Local\Temp\ipykernel_4468\3247654901.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_game_lineups = pd.read_csv("data-brazil/game_lineups.csv")


In [4]:
df_appearances = df_appearances.fillna("")
df_game_events = df_game_events.fillna("")
df_game_lineups = df_game_lineups.fillna("")
df_player_valuations = df_player_valuations.fillna("")
df_players = df_players.fillna("")
df_transfers = df_transfers.fillna("")
##
df_games = df_games.fillna("")
df_competitions = df_competitions.fillna("")
df_clubs = df_clubs.fillna("")
df_clubs_games = df_clubs_games.fillna("")

In [5]:
df_players.loc[df_players["player_id"] == 111819]

,player_id,first_name,last_name,name,last_season,current_club_id,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,height_in_cm,contract_expiration_date,current_club_domestic_competition_id,market_value_in_eur,highest_market_value_in_eur
462,111819,,Neto,Neto,2024,11,Brazil,Araxá,Brazil,1989-07-19 00:00:00,Goalkeeper,Goalkeeper,right,190.0,2025-06-30 00:00:00,GB1,2500000.0,18000000.0


In [6]:
df_transfers.loc[df_transfers["player_id"] == 111819]

,player_id,transfer_date,transfer_season,from_club_id,to_club_id,transfer_fee,market_value_in_eur
0,111819,2025-06-30,24/25,11,989,0.0,2500000.0
11,111819,2024-08-30,24/25,989,11,0.0,2500000.0
177,111819,2022-08-07,22/23,131,989,0.0,3000000.0
376,111819,2019-07-01,19/20,1049,131,26000000.0,18000000.0
459,111819,2017-07-07,17/18,506,1049,7000000.0,6000000.0
502,111819,2015-07-03,15/16,430,506,0.0,9000000.0


In [7]:
def transform_list(df):
  lista = list()
  for value in df.head(5).values:
    lista.append(dict(zip(df.columns, value)))
  return lista

In [8]:
def writeScheam(file, db, lista):
  with open(f"scheamNoSQL/{file}.txt", 'w', encoding='utf-8') as obj:
    obj.write(f"db.{db}.insertMany({lista})")

In [9]:
add_player = list()
for value in df_players.values:
  player = dict(zip(df_players.columns, value))
  game_events = transform_list(df_game_events.loc[df_game_events["player_id"] == value[0]])
  transfers = transform_list(df_transfers.loc[df_transfers["player_id"] == value[0]])
  add_player.append({**player, "game_events": game_events, "transfers": transfers})
writeScheam("players", "players", add_player)

In [10]:
add_clubs = list()
for value in df_clubs.values:
  clubs = dict(zip(df_clubs.columns, value))
  clubs_games = transform_list(df_game_events.loc[df_game_events["club_id"] == value[0]])
  games = transform_list(df_games.loc[df_games["home_club_id"] == value[0]])
  add_clubs.append({**clubs, "clubs_games": clubs_games, "games": games})
writeScheam("clubs", "clubs", add_clubs)

In [11]:
df_games

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_manager_name,away_club_manager_name,referee,home_club_formation,away_club_formation,aggregate,competition_type
0,2321044,L1,2013,2. Matchday,2013-08-18,16,23,2,1,Jürgen Klopp,Torsten Lieberknecht,Peter Sippel,4-2-3-1,4-3-2-1,2:1,domestic_league
1,2321060,L1,2013,3. Matchday,2013-08-25,23,24,0,2,Torsten Lieberknecht,Armin Veh,Wolfgang Stark,4-3-2-1,4-2-3-1,0:2,domestic_league
2,2321086,L1,2013,6. Matchday,2013-09-21,4,16,1,1,Michael Wiesinger,Jürgen Klopp,Knut Kircher,4-2-3-1,4-2-3-1,1:1,domestic_league
3,2321152,L1,2013,11. Matchday,2013-11-02,44,33,0,2,Jos Luhukay,Jens Keller,Günter Perl,4-2-3-1,4-2-3-1,0:2,domestic_league
4,2321205,L1,2013,17. Matchday,2013-12-21,41,39,2,3,Bert van Marwijk,Thomas Tuchel,Bastian Dankert,4-2-3-1,4-4-2 Diamond,2:3,domestic_league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56303,3852607,NL1,2022,12. Matchday,2022-10-30,383,467,3,0,Ruud van Nistelrooy,Rogier Meijer,Martin van den Kerkhof,4-3-3 Defending,4-3-3 Defending,3:0,domestic_league
56304,2589097,NL1,2015,23. Matchday,2016-02-14,467,383,0,3,Ernest Faber,Phillip Cocu,Kevin Blom,5-3-2,4-3-3 Attacking,0:3,domestic_league
56305,2700827,NL1,2016,23. Matchday,2017-02-18,383,467,3,1,Phillip Cocu,Peter Hyballa,Jochem Kamphuis,4-3-3 Attacking,5-3-2,3:1,domestic_league
56306,3852741,NL1,2022,27. Matchday,2023-04-01,467,383,2,4,Rogier Meijer,Ruud van Nistelrooy,Edwin van de Graaf,4-3-3 Defending,4-3-3 Defending,2:4,domestic_league


In [13]:
player

{'player_id': 1193867,
 'first_name': '',
 'last_name': 'Pedro Lima',
 'name': 'Pedro Lima',
 'last_season': 2024,
 'current_club_id': 543,
 'country_of_birth': 'Brazil',
 'city_of_birth': 'Cabedelo',
 'country_of_citizenship': 'Brazil',
 'date_of_birth': '2006-07-01 00:00:00',
 'sub_position': 'Right-Back',
 'position': 'Defender',
 'foot': 'right',
 'height_in_cm': 174.0,
 'contract_expiration_date': '2029-06-30 00:00:00',
 'current_club_domestic_competition_id': 'GB1',
 'market_value_in_eur': 5000000.0,
 'highest_market_value_in_eur': 5000000.0}

In [12]:
add_clubs = list()
for value in df_games.values:
  games = dict(zip(df_games.columns, value))
  game_lineups = transform_list(df_game_lineups.loc[df_game_lineups["club_id"] == value[3]])
  competitions = transform_list(df_competitions.loc[df_competitions["competition_id"] == value[0]])
  add_clubs.append({**games, "game_lineups": game_lineups, "competitions": competitions})
writeScheam("games", "games", add_clubs)